In [20]:
import os, time
import pandas as pd

##### 仅仅取每个news_side_id的唯一值
##### 5分大约9公里，15分为我27公里，因为坐标过于细致，用2位小数来区分，两位小数约是1公里，三位111m，四位11m
- 这样这里需要去掉一些重复坐标点

In [16]:
def statisticsData(data, filepath, filename, txtfilename):

    unique_sites = data.drop_duplicates(subset='new_site_id', keep='first', inplace=False).reset_index(drop=True)
    unique_sites.loc[:, 'lat_wgs84'] = unique_sites['lat_wgs84'].apply(lambda x: f"{x:.2f}") # .round(2)
    unique_sites.loc[:, 'lon_wgs84'] = unique_sites['lon_wgs84'].apply(lambda x: f"{x:.2f}")  # .round(2)
    # Group by the rounded 'lat_wgs84' and 'lon_wgs84' and take the first occurrence
    unique_sites_grouped = unique_sites.groupby(['lat_wgs84', 'lon_wgs84']).first().reset_index(drop=False)
    unique_sites_grouped = unique_sites_grouped.sort_values(by='new_site_id')
    unique_sites_grouped['new_new_site_id'] = range(1, len(unique_sites_grouped) + 1)
    # Save the unique sites to a new Excel file
    unique_sites_grouped.to_csv(os.path.join(filepath, filename + '.csv'), index=True)
    # Select the columns to save to the text file
    lat_lon_data = unique_sites_grouped[['lat_wgs84', 'lon_wgs84']]
    # Save the DataFrame to a text file, without headers and index
    lat_lon_data.to_csv(os.path.join(filepath, txtfilename + '.txt'), header=False, index=False, sep='\t')

## load WQI file 
### extract year of 1978-2017

In [18]:
def main():
    os.chdir(r'F:/WQI20231129')
    Inputpath = 'MetaData'
    Outputpath = 'MetaData/unique_sites'
    filename = 'meta_TN_GRQA.xlsx'
    Syear = 1978
    Eyear = 2017
    OriData = pd.read_excel(os.path.join(Inputpath, filename), engine = 'openpyxl', nrows=500, header=0, index_col=0)
    TarData = OriData[(OriData['obs_year'] >= Syear) & (OriData['obs_year'] <= Eyear)].reset_index(drop=True)
    # TarData.to_excel('TarData.xlsx', index=True)
    statisticsData(TarData, Outputpath, 'unique_sites', 'coordinate')

In [17]:
if __name__ == '__main__':
    start_time = time.time()
    main()
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f'Time taken: {elapsed_time: .2f}seconds')

Time taken:  0.21seconds
